In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import graphviz
import openml
from sklearn import impute, tree, pipeline
from sklearn.tree import DecisionTreeClassifier


In [11]:

# Define a scikit-learn classifier or pipeline
tree = DecisionTreeClassifier()

# Download the OpenML task for the german credit card dataset with 10-fold
# cross-validation.
task = openml.tasks.get_task(32)

run = openml.runs.run_model_on_task(tree, task)

#print(task)

OpenMLServerException: https://www.openml.org/api/v1/xml/flow/exists returned code 102: No authentication (Please provide API key for all requests other than HTTP GET) - None